# Oppgave 21 - Salmon Lice

In [85]:
import pandas as pd
salmonLice = pd.read_csv('SalmonLice.csv', sep=';', decimal=',')
salmonLice.head()

,SalmonLice,Farm,CleanerFish
0,1.753024,Farm 1,Lumpfish
1,1.086809,Farm 1,Lumpfish
2,1.185394,Farm 1,Lumpfish
3,1.583848,Farm 1,Lumpfish
4,1.587465,Farm 1,Lumpfish


## oppgave 21c)

In [86]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

##### Får ikke ut som en vanlig nested model i anova, så legger sammen interaction og CleanerFish for å få Betta_j(i)

#### Tar utgangspunkt i at begge er fixed

In [87]:
model = ols('SalmonLice ~ C(CleanerFish, Sum)*C(Farm, Sum)', data = salmonLice).fit()
anova = sm.stats.anova_lm(model)
anova

,df,sum_sq,mean_sq,F,PR(>F)
"C(CleanerFish, Sum)",2.0,14.915356,7.457678,56.025682,1.403649e-12
"C(Farm, Sum)",1.0,0.219742,0.219742,1.650811,2.058895e-01
"C(CleanerFish, Sum):C(Farm, Sum)",2.0,1.252197,0.626098,4.703554,1.434658e-02
Residual,42.0,5.590695,0.133112,NaN,NaN


In [88]:
ss_betta_ji = anova['sum_sq'][1] + anova['sum_sq'][2]
df_betta_ji = anova['df'][1] + anova['df'][2]
ms_betta_ji = ss_betta_ji/df_betta_ji
f0_betta_ji = ms_betta_ji/anova['mean_sq'][3]

In [89]:
print(f'ss: {ss_betta_ji}\nms: {ms_betta_ji}\ndf: {df_betta_ji}\nf0_betta: {f0_betta_ji}')

ss: 1.4719393586719451
ms: 0.4906464528906484
df: 3.0
f0_betta: 3.685973253898773


In [90]:
model_nested = ols('SalmonLice ~ C(CleanerFish, Sum) + C(Farm, Sum):C(CleanerFish, Sum)', data = salmonLice).fit()
anova_nested = sm.stats.anova_lm(model_nested)
anova_nested

,df,sum_sq,mean_sq,F,PR(>F)
"C(CleanerFish, Sum)",2.0,14.915356,7.457678,56.025682,1.403649e-12
"C(Farm, Sum):C(CleanerFish, Sum)",3.0,1.471939,0.490646,3.685973,1.917547e-02
Residual,42.0,5.590695,0.133112,NaN,NaN


In [91]:
import researchpy as rp
salmonLice['Farm:CleanerFish'] = salmonLice.Farm + ':' + salmonLice.CleanerFish
summary_cont = rp.summary_cont(salmonLice.groupby(['CleanerFish', 'Farm'])['SalmonLice'])
summary_cont

N    Mean      SD      SE  95% Conf.  Interval
CleanerFish Farm                                                  
Control     Farm 1  8  2.0872  0.4590  0.1623     1.7034    2.4709
            Farm 2  8  2.3805  0.3628  0.1283     2.0772    2.6838
Lumpfish    Farm 1  8  1.4424  0.2400  0.0849     1.2417    1.6431
            Farm 2  8  1.1275  0.3006  0.1063     0.8761    1.3788
Wrappers    Farm 1  8  0.6953  0.3519  0.1244     0.4012    0.9895
            Farm 2  8  1.1228  0.4296  0.1519     0.7637    1.4820

In [92]:
mu = salmonLice['SalmonLice'].mean()
tau_1 = (summary_cont['Mean'][0]+summary_cont['Mean'][1])/2 - mu
tau_2 = (summary_cont['Mean'][2]+summary_cont['Mean'][3])/2 - mu
tau_3 = (summary_cont['Mean'][4]+summary_cont['Mean'][5])/2 - mu
print(f'Mu: {mu}\nTau1: {tau_1}\nTau2: {tau_2}\nTau3: {tau_3}')

Mu: 1.4759473706041673
Tau1: 0.7579026293958331
Tau2: -0.19099737060416744
Tau3: -0.5668973706041672


In [93]:
model_nested.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             SalmonLice   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     24.62
Date:                Tue, 25 Aug 2020   Prob (F-statistic):           1.70e-11
Time:                        15:19:36   Log-Likelihood:                -16.507
No. Observations:                  48   AIC:                             45.01
Df Residuals:                      42   BIC:                             56.24
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================
                                                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                  1.4759      0.053     28.027      0.000       1.370       1.582
C(CleanerFish, Sum)[S.Control]                             0.7579      0.074     10.177      0.000       0.608       0.908
C(CleanerFish, Sum)[S.Lumpfish]                           -0.1910      0.074     -2.565      0.014      -0.341      -0.041
C(Farm, Sum)[S.Farm 1]:C(CleanerFish, Sum)[mean]          -0.0677      0.053     -1.285      0.206      -0.174       0.039
C(Farm, Sum)[S.Farm 1]:C(CleanerFish, Sum)[S.Control]     -0.0790      0.074     -1.061      0.295      -0.229       0.071
C(Farm, Sum)[S.Farm 1]:C(CleanerFish, Sum)[S.Lumpfish]     0.2251      0.074      3.023      0.004       0.075       0.375
==============================================================================
Omnibus:                        1.769   Durbin-Watson:                   2.207
Prob(Omnibus):                  0.413   Jarque-Bera (JB):                1.184
Skew:                           0.044   Prob(JB):                        0.553
Kurtosis:                       2.236   Cond. No.                         1.73
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## oppg 21d)

In [94]:
model = ols('SalmonLice ~ C(Farm, Sum)*C(CleanerFish, Sum)', data = salmonLice).fit()
anova = sm.stats.anova_lm(model)
anova

,df,sum_sq,mean_sq,F,PR(>F)
"C(Farm, Sum)",1.0,0.219742,0.219742,1.650811,2.058895e-01
"C(CleanerFish, Sum)",2.0,14.915356,7.457678,56.025682,1.403649e-12
"C(Farm, Sum):C(CleanerFish, Sum)",2.0,1.252197,0.626098,4.703554,1.434658e-02
Residual,42.0,5.590695,0.133112,NaN,NaN


## oppg 21e)

In [95]:
model_nested = ols('SalmonLice ~ C(CleanerFish, Sum) + C(Farm, Sum):C(CleanerFish, Sum)', data = salmonLice).fit()
anova_nested = sm.stats.anova_lm(model_nested)
anova_nested

,df,sum_sq,mean_sq,F,PR(>F)
"C(CleanerFish, Sum)",2.0,14.915356,7.457678,56.025682,1.403649e-12
"C(Farm, Sum):C(CleanerFish, Sum)",3.0,1.471939,0.490646,3.685973,1.917547e-02
Residual,42.0,5.590695,0.133112,NaN,NaN


##### beregner en ny F0 til cleanerFish fordi Farm er random.....

In [104]:
import scipy.stats as stats
alpha = 0.05
ms_a = anova_nested['mean_sq'][0]
ms_ba = anova_nested['mean_sq'][1]
df_a = anova_nested['df'][0]
df_ba = anova_nested['df'][1]
F0 = ms_a/ms_ba
f_alpha = stats.f.isf(alpha, df_a, df_ba)
print(f'F0: {F0}\nf_alpha: {f_alpha}\nRejcet: {F0>f_alpha}')

F0: 15.19969833073694
f_alpha: 9.552094495921155
Rejcet: True


## oppg 21f) - er lump og wrappers bedre enn kontroll??? 

In [108]:
mu = salmonLice['SalmonLice'].mean()
tau_control = (summary_cont['Mean'][0]+summary_cont['Mean'][1])/2 - mu
tau_lumpfish = (summary_cont['Mean'][2]+summary_cont['Mean'][3])/2 - mu
tau_wrappers = (summary_cont['Mean'][4]+summary_cont['Mean'][5])/2 - mu
print(f'Mu: {mu}\nTau_control: {tau_control}\nTau_lumpfish: {tau_lumpfish}\nTau_wrappers: {tau_wrappers}')

Mu: 1.4759473706041673
Tau_control: 0.7579026293958331
Tau_lumpfish: -0.19099737060416744
Tau_wrappers: -0.5668973706041672


In [119]:
import scipy.stats as stats
alpha = 0.05
c1 = 0.5
c2 = 0.5
c3 = -1
b = 2
n = 8
gamma_hatt = (tau_lumpfish*c1)+(tau_wrappers*c2)+(tau_control*c3)
ms_ba = anova_nested['mean_sq'][1]
se_gamma_hatt = ((ms_ba/(b*n))*(c1**2+c2**2+c3**2))**0.5
t_alpha = stats.t.isf(alpha/2, df_ba)
t0 = gamma_hatt/se_gamma_hatt
print(f'T0: {t0}\nt_alpha: {t_alpha}\nReject: {abs(t0)>t_alpha}')

T0: -5.300699642645096
t_alpha: 3.1824463052842638
Reject: True


## oppg 21h) 
##### vi ser allerede på anova resultatet at vi kan forkaste H0 med alpha=0.05. Dermed er det egentlig ikke vits å gjøre testen, men kan gjøre det for 'morroksyld' (hvis jeg regner det som random, så må det gjøres endringer i tiligere oppgaver også (oppg c og e))

In [120]:
anova_nested

,df,sum_sq,mean_sq,F,PR(>F)
"C(CleanerFish, Sum)",2.0,14.915356,7.457678,56.025682,1.403649e-12
"C(Farm, Sum):C(CleanerFish, Sum)",3.0,1.471939,0.490646,3.685973,1.917547e-02
Residual,42.0,5.590695,0.133112,NaN,NaN


In [121]:
alpha = 0.05
ms_ba = anova_nested['mean_sq'][1]
ms_e = anova_nested['mean_sq'][2]
df_ba = anova_nested['df'][1]
df_e = anova_nested['df'][2]
f_alpha = stats.f.isf(alpha, df_ba, df_e)
f0 = ms_ba/ms_e
print(f'F0: {f0}\nf_alpha: {f_alpha}\nReject: {abs(f0)>f_alpha}')

F0: 3.685973253898773
f_alpha: 2.8270487120861265
Reject: True


# OPPGAVE 22

## oppg 22b)

#### Tar utgangspunkt i at Tau og betta er random

In [3]:
import pandas as pd
import statsmodels.api as sm 
from statsmodels.formula.api import ols
ex14 = pd.read_csv('EX14_1Montg_data.csv', sep=';', decimal=',')

In [4]:
model_test = ols('Purity ~ C(Supplier, Sum) + C(Supplier, Sum):C(Batch, Sum)', data = ex14).fit()
anova_test = sm.stats.anova_lm(model_test)
anova_test

,df,sum_sq,mean_sq,F,PR(>F)
"C(Supplier, Sum)",2.0,15.055556,7.527778,2.852632,0.077363
"C(Supplier, Sum):C(Batch, Sum)",9.0,69.916667,7.768519,2.943860,0.016674
Residual,24.0,63.333333,2.638889,NaN,NaN


#### Siden betta og tau er random, så må vi regne ut F til tau på nytt, beregne sigma til betta og sigma til tau.
##### I første omgang beregner vi bare sigmaene og mu

In [6]:
n = 3
a = 3
b = 4
mu_ex = ex14['Purity'].mean()
sigma2 = anova_test['mean_sq'][2]
sigma2_ba = (anova_test['mean_sq'][1]-anova_test['mean_sq'][2])/n
sigma2_a = (anova_test['mean_sq'][0] - anova_test['mean_sq'][1])/(b*a)
print(f'Mu: {mu_ex}\nsigma2: {sigma2}\nsigma2_b(a): {sigma2_ba}\nsimga2_a: {sigma2_a}')

Mu: 0.3611111111111111
sigma2: 2.638888888888889
sigma2_b(a): 1.7098765432098766
simga2_a: -0.020061728395061856


In [10]:
import scipy.stats as stats
stats.f.isf(0.975,8,9)

0.2295034452119975